In [3]:
#for transfer leanring we need to change the final outlayers
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
#loading the image
from tensorflow.keras.preprocessing import image
#data augmentations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#initializing the image size
img_size=[224,224]

In [5]:
#data path
data_path='/content/drive/MyDrive/archive (16)/Food Classification dataset'

In [6]:
#using the imagedatagenerator to make some augmentations
datagen=ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2,rotation_range=50,zoom_range=0.5,horizontal_flip=True,vertical_flip=True)
datagen

In [7]:
#loading the train data
train_generator=datagen.flow_from_directory(directory=data_path,target_size=img_size,batch_size=32,class_mode='categorical',subset='training',shuffle=True)
train_generator

Found 3242 images belonging to 10 classes.


In [8]:
train_generator.class_indices

{'kulfi': 0,
 'masala_dosa': 1,
 'momos': 2,
 'omelette': 3,
 'paani_puri': 4,
 'pakode': 5,
 'pav_bhaji': 6,
 'pizza': 7,
 'samosa': 8,
 'sushi': 9}

In [9]:
train_generator.samples

3242

In [10]:
#loading the test data
val_generator=datagen.flow_from_directory(directory=data_path,target_size=img_size,batch_size=32,class_mode='categorical',subset='validation',shuffle=False)
val_generator

Found 805 images belonging to 10 classes.


In [11]:
val_generator.class_indices

{'kulfi': 0,
 'masala_dosa': 1,
 'momos': 2,
 'omelette': 3,
 'paani_puri': 4,
 'pakode': 5,
 'pav_bhaji': 6,
 'pizza': 7,
 'samosa': 8,
 'sushi': 9}

In [12]:
val_generator.samples

805

In [13]:
model1=ResNet50(weights='imagenet',include_top=False,input_shape=img_size+[3])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [14]:
model1.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [15]:
#freezing the base layers
for layer in model1.layers:
  layer.trainable=False

In [16]:
model1.output.shape  #in default the cnn models output layer is 3d (single image )and it will be have like 4d because of batch

(None, 7, 7, 2048)

In [17]:
#customizing the classifier on the top
x=Flatten()(model1.output)   #flattening the output layer
#reason the dense layer requires only in the vextor (1d array form)

In [18]:
x.shape

(None, 100352)

In [19]:
#output layer
prediction=Dense(len(train_generator.class_indices),activation='softmax')(x)
prediction

<KerasTensor shape=(None, 10), dtype=float32, sparse=False, ragged=False, name=keras_tensor_176>

In [20]:
model=Model(inputs=model1.input,outputs=prediction)

In [21]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,591,242 (93.81 MB)

 Trainable params: 1,003,530 (3.83 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [22]:
model.output.shape

(None, 10)

In [23]:
#compiling
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#fitting the model
fit=model.fit(train_generator,validation_data=val_generator,epochs=15,steps_per_epoch=len(train_generator),validation_steps=len(val_generator))

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 1076s 10s/step - accuracy: 0.5020 - loss: 13.2982 - val_accuracy: 0.7764 - val_loss: 4.7804
Epoch 2/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 753s 7s/step - accuracy: 0.7503 - loss: 5.8602 - val_accuracy: 0.7888 - val_loss: 5.1169
Epoch 3/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 718s 7s/step - accuracy: 0.8196 - loss: 4.4483 - val_accuracy: 0.8335 - val_loss: 4.6856
Epoch 4/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 743s 7s/step - accuracy: 0.8321 - loss: 4.4166 - val_accuracy: 0.8398 - val_loss: 4.7895
Epoch 5/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 709s 7s/step - accuracy: 0.8528 - loss: 4.0850 - val_accuracy: 0.8385 - val_loss: 5.6312
Epoch 6/15
 40/102 ━━━━━━━━━━━━━━━━━━━━ 6:01 6s/step - accuracy: 0.8397 - loss: 4.8525

In [ ]:
model.save('/content/drive/MyDrive/food_model(resnet).keras')

# ***Fine Tuning***

In [ ]:
model1.trainable=True

In [ ]:
for layer in model1.layers[:-5]:  # adjust according to layer count
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(1e-5),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
fit2= model.fit(train_generator,validation_data=val_generator,epochs=15,steps_per_epoch=len(train_generator),validation_steps=len(val_generator)) # fine-tuning, usually fewer epochs

In [ ]:
len(model1.layers)